In [4]:
import numpy as np
import scipy.stats.qmc as qmc
import matplotlib.pyplot as plt
import numpy.random as random

import scipy as sp
import scipy.integrate as integrate

from tqdm import tqdm 

GENERATOR = np.random.default_rng(seed=None)

# Exericse sheet 8

## Exercise 4 (Two simple QMC test problems)

In [5]:
### FINITE ELEMENT STUFF

def kappa(
    x: float,  # Evaluation point
    q: float,
    omegas: np.array,
) -> float:
    kappa = np.zeros(len(x))
    for j in range(1, len(omegas) + 1):
        # Compute corresponding basis function.
        a = (1 / j ** q) * np.sin(2 * j * np.pi * x)
        kappa += omegas[j - 1] * a
    
    return np.exp(kappa)

def create_stiffness_matrix(
    N: int,
    q: float,
    omegas: np.array,
):
    h = 1 / (N - 1)
    diagonals = [np.empty((N - 1)), np.empty((N)), np.empty((N - 1))]
    TRAPEZ_POINTS = 21
    for i in range(len(diagonals[1])):
        #diagonals[1][i] = 1/(h**2) * integrate.quad(lambda x: kappa(x, q, omegas), h*(i-1), h*(i+1))[0]
        x = np.linspace(h*(i-1), h*(i+1),TRAPEZ_POINTS)
        y = kappa(x, q,  omegas)
        diagonals[1][i]  =    np.trapz(y, x)
    for i in range(len(diagonals[0])):
        #diagonals[0][i] = -1/(h**2) * integrate.quad(lambda x: kappa(x, q, omegas), h*(i), h*(i+1))[0]
       
        x = np.linspace( h*(i), h*(i+1),TRAPEZ_POINTS)
        y = kappa(x, q, omegas)
        diagonals[0][i]  =    np.trapz(y, x)

        diagonals[2][i] = diagonals[0][i]
    # Impose boundary conditions.
    diagonals[0][0] = 0
    diagonals[2][0] = 0
    diagonals[0][-1] = 0
    diagonals[2][-1] = 0
    
    mat = sp.sparse.diags(diagonals, [-1, 0, 1])

    return mat


def create_rhs(N: int, f):
    h = 1 / (N - 1)
    TRAPEZ_POINTS = 21
    vec = np.empty((N))

    for i in range(len(vec)):
        #vec[i] = integrate.quad(lambda x: f(x)*(1-abs(h*i-x)/h), h*(i-1), h*(i+1))[0]
        x = np.linspace(h*(i-1), h*(i+1),TRAPEZ_POINTS)
        y = f(x)*(1-abs(h*i-x)/h)
        vec[i] =    np.trapz(y, x)
    # Impose boundary conditions.
    vec[0] = 0
    vec[-1] = 0

    return vec



In [6]:
###GENERATING VECTORS FOR LATTICE RULE

def generating_vectors(s, N=None):
    # Source: https://wsc.project.cwi.nl/woudschoten-conferences/2016-woudschoten-conference/FKtalk1.pdf, Slide 14
    if s==2:
        return np.array([1, 19])
    if s==10:# Source: https://people.cs.kuleuven.be/~dirk.nuyens/qmc-generators/LATSEQ/exew_base2_m20_a3_HKKN.txt
        return np.array([1, 364981, 245389, 97823, 488939, 62609, 400749, 385317, 21281, 223487])
    else:
        return generate_custom_vectors(N,s)

def generate_custom_vectors(N,s):
    
    def B_2(x):
        return np.power(x,2)-x + (1/6)
    
    U_N = np.gcd(np.arange(0,N), N)
    U_N = np.arange(N)[U_N==1]
    z = np.empty(s)
    p = np.ones(N)
    #print("U_N",U_N)
     
    G_N = B_2(np.modf(np.outer(U_N,np.arange(N))/N )[0])
    #print("G_N",G_N )
    #print("GN SHAPE",G_N.shape)
    #print("sum",np.sum(G_N,axis=0))
    #print("sum SHAPE",np.sum(G_N,axis=0).shape)
    z[0]=1
      
    for i in range(1,s):
        #print("p:",p)
        delta = (1/N)*np.inner(G_N,p )
        #print("delta",delta.shape)
        #print("argmin delta",np.argmin(delta))
        z[i]= U_N[np.argmin(delta)]
        p = np.multiply((np.ones(N) + G_N[i]),p)
    return z

#generate_custom_vectors(64,2)

In [7]:
## LATTICE POINTS
def shift_points(s: int, points: np.array):
    shift_vector = np.random.uniform(0, 1, s)
        
    return np.modf(points + shift_vector)[0]-0.5

def generate_lattice_points(
    z,  # generating vector
    m,  # 2 ** m number of points
    shift = False,
    previous_lattice=np.array([]),  # reuse points from previous lattice
    
):
    
    
    def generate(index):
        return np.modf(
            np.reshape(index, (-1, 1)) * z / N
        )[0]

    # Set number of points.
    N = 2 ** m
    
    rows = np.arange(0, N, 1)

    if previous_lattice.size == 0:
        # Generate first lattice.
        new_lattice = generate(rows)
    else:
        new_lattice = np.empty([N, z.size])
        
        even_rows = rows[rows % 2 == 0]
        odd_rows = rows[rows % 2 == 1]
        
        new_lattice[even_rows] = previous_lattice
        new_lattice[odd_rows] = generate(odd_rows)
    
   
    return new_lattice 



In [8]:
### SOBOL POINTS

def generate_sobol_points(
    s: int,
    m: int,
    scramble=True,
    generator=None
):
    sampler = qmc.Sobol(d=s, scramble=scramble,seed=generator.integers(10**8))
    return sampler.random_base2(m=m) - 0.5

In [11]:
### MONTE CARLO

def MC(s,q,N_samples,h,sample_Var=False):
     
    N = int(1/h) +1 
    MC_samples = np.empty(N_samples)
     
    
    f = create_rhs(N, lambda x: 1)                    
    for j in tqdm(range(N_samples)):
        omegas = random.uniform(-0.5,0.5,s)
        mat = create_stiffness_matrix(N, q, omegas)

        u = sp.sparse.linalg.spsolve(mat, f)
        MC_samples[j] = u[int((N - 1) * 0.7)]
    MC_mean  = np.mean(MC_samples)

    if sample_Var:
        MC_var  = np.var(MC_samples,ddof=1)                  
        return MC_mean,MC_var
    else:
        return MC_mean
    

In [12]:
## QUASI-MONTE CARLO

def QMC(s,q,m,h,R,sobol=True,sample_Var=False):
 

 
    N_samples = 2**m
    MC_mean = np.empty(shape=R)
    MC_var= np.empty(shape=R)

    if not sobol:
        z = generating_vectors(s, N=2**m)
        l_points =generate_lattice_points(z,m)

    N = int(1/h) +1 
    f = create_rhs(N, lambda x: 1)

    for i in tqdm(range(R)):
        
        if sobol:
            points = generate_sobol_points(s,m,scramble=True,generator=GENERATOR)
        else:
            points = shift_points(s,l_points)
            


        
        MC_samples = np.empty(N_samples)
        
        
                            
        for j in range(N_samples):
            omegas =  points[j]
            mat = create_stiffness_matrix(N, q, omegas)

            u = sp.sparse.linalg.spsolve(mat, f)
            MC_samples[j] = u[int((N - 1) * 0.7)]
        MC_mean[i] = np.mean(MC_samples)
        if sample_Var:
            MC_var[i]  = np.var(MC_samples,ddof=1)
                
    if sample_Var:    
        return np.mean(MC_mean),np.mean(MC_var)
    else:
        return np.mean(MC_mean)


a) TESTING THE FUNCTIONS

In [13]:
R=8
m=10
N_samples=(2**m) * R
MC_res,MC_svar = MC(s=10,q=2,N_samples=N_samples,h=0.05,sample_Var=True)
lattice_res,lattice_svar = QMC(s=10,q=2,m=m,R=R,h=0.05,sobol=False,sample_Var=True)
sobol_res,sobol_svar = QMC(s=10,q=2,m=m,R=R,h=0.05,sobol=True,sample_Var=True)


  0%|          | 0/8192 [00:00<?, ?it/s]C:\Users\Aaron Maekel\AppData\Local\Temp\ipykernel_7084\4031945572.py:14: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = sp.sparse.linalg.spsolve(mat, f)
  0%|          | 0/8 [00:00<?, ?it/s]C:\Users\Aaron Maekel\AppData\Local\Temp\ipykernel_7084\3147889133.py:29: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = sp.sparse.linalg.spsolve(mat, f)
100%|██████████| 8/8 [00:22<00:00,  2.82s/it]


In [16]:
print("MC ESTIMATOR:                   ",MC_res," SAMPLE VARIANCE: ",MC_svar)
print("QMC ESTIMATOR with lattice rule:",lattice_res," SAMPLE VARIANCE: ",lattice_svar)
print("QMC ESTIMATOR with sobol rule:  ",sobol_res," SAMPLE VARIANCE: ",sobol_svar)

MC ESTIMATOR:                    0.010953589642425064  SAMPLE VARIANCE:  0.0052880905544996255
QMC ESTIMATOR with lattice rule: 0.010459284869291666  SAMPLE VARIANCE:  0.005345585363143194
QMC ESTIMATOR with sobol rule:   0.010469506352836546  SAMPLE VARIANCE:  0.005351991767562446


b) ERROR OF CONVERGENCE RATE

In [ ]:
R=8
mmesh =np.arange(10,20)
q=2
N_samples=(2**m) * R
MC_res = np.empty(len(mmesh))
MC_svar  = np.empty(len(mmesh))
lattice_res = np.empty(len(mmesh))
lattice_svar = np.empty(len(mmesh))
sobol_res = np.empty(len(mmesh))
sobol_svar = np.empty(len(mmesh))
i=0
for m in mmesh:

    MC_res[i], MC_svar[i] = MC(s=10,q=q,N_samples=N_samples,h=0.05,sample_Var=True)
    lattice_res[i],lattice_svar[i] = QMC(s=10,q=q,m=m,R=R,h=0.05,sobol=False,sample_Var=True)
    sobol_res[i],sobol_svar[i] = QMC(s=10,q=q,m=m,R=R,h=0.05,sobol=True,sample_Var=True)
    i+=1

  0%|          | 0/8192 [00:00<?, ?it/s]C:\Users\Aaron Maekel\AppData\Local\Temp\ipykernel_7084\4031945572.py:14: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = sp.sparse.linalg.spsolve(mat, f)
  0%|          | 0/8 [00:00<?, ?it/s]C:\Users\Aaron Maekel\AppData\Local\Temp\ipykernel_7084\3147889133.py:29: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = sp.sparse.linalg.spsolve(mat, f)
 12%|█▎        | 1/8 [00:11<01:18, 11.16s/it]

In [17]:
import matplotlib.pyplot as plt
 

 
 
# Plot the curves
plt.figure(figsize=(8, 6))
plt.plot(R, lattice_sample_var, label="lattice rule", color='blue')
plt.plot(R,sobol_sample_var, label="sobol rule", color='red')
#plt.plot(R, y3, label="tan(x)", color='green')



# Adding labels and title
plt.xlabel("R")
plt.ylabel("Sample Var")
plt.title("Three Different Sample variances depending on amount of realizations R:  ")

# Show legend
plt.legend()

# Display the plot
plt.yscale("log")
plt.grid(True)
plt.show()


NameError: name 'lattice_sample_var' is not defined

<Figure size 800x600 with 0 Axes>